In [28]:
import pandas as pd
from IPython.display import display
from datetime import datetime, timedelta
from GSPC_model import process_data_and_backtest
import yfinance as yf

pd.options.display.float_format = '{:.2f}'.format

In [29]:
def simulate_trading_algorithm(starting_balance, data, index_date):
    current_balance = starting_balance
    shares_held = 0
    cost_basis = 0

    trading_log = []

    for i, current_date in enumerate(index_date[:-1]):
        # Extract relevant data
        row = data.loc[current_date]
        prediction = row['Predictions_SP500']
        vfiax_close = row['VFIAX_close']

        # Buy
        if prediction == 1 and shares_held == 0:
            shares_bought = current_balance / vfiax_close
            cost_basis = current_balance
            shares_held += shares_bought
            current_balance = 0

            trading_log.append({
                'Date': current_date,
                'Prediction': prediction,
                'Decision': 'Buy',
                'Shares': shares_bought,
                'Price': vfiax_close,
                'Profit/Loss': 0,  
                'Current Balance': current_balance
            })

        # Hold
        elif prediction == 1 and shares_held > 0:
            current_balance = shares_held * vfiax_close

            trading_log.append({
                'Date': current_date,
                'Prediction': prediction,
                'Decision': 'Hold',
                'Shares': shares_held,
                'Price': vfiax_close,
                'Profit/Loss': 0,
                'Current Balance': current_balance
            })

        # Sell
        elif prediction == 0 and shares_held > 0:
            current_balance = shares_held * vfiax_close
            profit_loss = current_balance - cost_basis
            shares_held = 0
            cost_basis = 0

            trading_log.append({
                'Date': current_date,
                'Prediction': prediction,
                'Decision': 'Sell',
                'Shares': 0,
                'Price': vfiax_close,
                'Profit/Loss': profit_loss,
                'Current Balance': current_balance
            })

    trading_log_df = pd.concat([pd.DataFrame([x]) for x in trading_log], ignore_index=True)

    return current_balance, trading_log_df

In [30]:
vfiax = yf.Ticker("VFIAX")
vfiax = vfiax.history(start="1990-01-02", end=None)

sp500 = process_data_and_backtest()

sp500 = pd.merge(sp500, vfiax['Close'].rename('VFIAX_close'), left_index=True, right_index=True, how='left')

result, trading_log = simulate_trading_algorithm(10000, sp500, sp500.index)

print("Final balance:", result)

print("\nTrading Log:")
display(trading_log)

Precision: 0.7783641160949868
Value Counts of Predictions:
Predictions
0.00    1606
1.00     758
Name: count, dtype: int64
Final balance: 1023813.8507464501

Trading Log:


,Date,Prediction,Decision,Shares,Price,Profit/Loss,Current Balance
0,2014-07-31 00:00:00-04:00,1.00,Buy,65.95,151.63,0.00,0.00
1,2014-08-01 00:00:00-04:00,1.00,Hold,65.95,151.19,0.00,9971.40
2,2014-08-04 00:00:00-04:00,0.00,Sell,0.00,152.28,43.18,10043.18
3,2014-08-05 00:00:00-04:00,1.00,Buy,66.59,150.82,0.00,0.00
4,2014-08-06 00:00:00-04:00,1.00,Hold,66.59,150.87,0.00,10046.58
...,...,...,...,...,...,...,...
1148,2023-11-29 00:00:00-05:00,1.00,Buy,2409.49,420.89,0.00,0.00
1149,2023-11-30 00:00:00-05:00,1.00,Hold,2409.49,422.61,0.00,1018274.86
1150,2023-12-01 00:00:00-05:00,0.00,Sell,0.00,425.13,10216.22,1024346.83
1151,2023-12-04 00:00:00-05:00,1.00,Buy,2422.60,422.83,0.00,0.00
